### Import the Starting Databases(Javier)

In [ ]:
import numpy as np #we use numpy to create arrays and mathematical process
import sep #sep is a source exraction and photometry library 

## Import FITS, matplotlib, set figure parameters

In [ ]:
from astropy.io import fits #We use "fits" to store images and tables 
import matplotlib.pyplot as plt #Allows us to show the images taken from the data
from matplotlib import rcParams #rcParams is used to set color bar size, color,shape,origin,axis.


%matplotlib inline 

rcParams['figure.figsize'] = [10., 8.] #default size of figures for the notebook 

## Read the image into a 2-D array 

In [ ]:
fname = "original image.fits" #create a new name file 
hdu_list = fits.open(fname) #gives us header names and data
hdu_list.info() #summary of the opened fits file

## Get dimensions of image(Aakash)

In [ ]:
image_data = fits.getdata(fname) #data from the image 
print(type(image_data)) #prints the type of data 
print(image_data.shape) #prints the data shape in x,y plane

## Show image data

In [ ]:
plt.imshow(image_data, cmap='gray', origin = 'lower') #displays the image from the resultant array data
plt.colorbar() #displays color bar on right side of the image

## Additional data(Tali) 

In [ ]:
print('Min:', np.min(image_data)) #prints the minimum value in the data array 
print('Max:', np.max(image_data)) #prints the maximum value in the data array 
print('Mean:', np.mean(image_data)) #prints the mean value in the data array 
print('Stdev:', np.std(image_data)) #prints the standard deviation of the values in the data array 

## Get a better quality image using LogNorm(Aakash)

In [ ]:
from matplotlib.colors import LogNorm #normailization of  given value in range of 0-1 in log scale

In [ ]:
plt.imshow(image_data, cmap='gray', norm=LogNorm(), origin = 'lower') #image is created with a gray color map, normalized to a log scale

cbar = plt.colorbar(ticks=[5.e3,1.e4,2.e4]) #creates a color bar on the right hand side
cbar.ax.set_yticklabels(['5,000', '10,000','20,000']) #y-axes labled at 5000,10000,20000

plt.savefig('fig1.png',bbox_inches='tight', dpi=300) #We now can save the figure in PNG format.

In [ ]:
bkg = sep.Background(image_data) #create an array of the background using the default settings

In [ ]:
print(bkg.globalback) #prints the global mean of the image background
print(bkg.globalrms) #prints the global noise of the image background

In [ ]:
bkg_image = bkg.back() #set a 2-d array of background data, same size as the original image

In [ ]:
plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
#interpolation = 'nearest' means pixels are shown as a square of multiple pixels - works well when small images are scaled up.
#cmap = 'gray' uses gray color map. 
#origin ='lower' sets the origin to the lower left hand corner
plt.colorbar(); #displays the color bar
plt.savefig('fig2.png',bbox_inches='tight', dpi=300) #save the figure as a PNG in the local folder. dpi will change during rasterizing so set bbox to tight.

In [ ]:
bkg_rms = bkg.rms() #set the background error (root mean square error)

In [ ]:
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower') 
#we used interpolation=nearest to help refine the small image that was scaled. 
#cmap = 'gray' uses gray color map. 
#origin ='lower' origin is placed in the lower lefthand side
plt.colorbar(); #displays the color bar
plt.savefig('fig3.png',bbox_inches='tight', dpi=300) #save the figure as a PNG, but use 'tight' for same reasons as above.

In [ ]:
data_sub = image_data - bkg #separates the background from the image to acquire new analytical data

### Detect objects on the background-subtracted data(Javier)

In [ ]:
objects = sep.extract(data_sub, 1.5, err=bkg.globalrms) #new data is extracted from the object

In [ ]:
number_of_objects = len(objects) #number_of_objects is set to the length of the object list
print(number_of_objects) #prints length of object list


In [ ]:
# Ellipse module gives us the capability of drawing ellipse around the potential objects in the image
from matplotlib.patches import Ellipse 

#backgroung image subtracted and plotted
fig, ax = plt.subplots() #gives us the ability to combine multiple plots on a figure with proper axes
m, s = np.mean(data_sub), np.std(data_sub) #we use the data_sub array elements to  find the mean and standard deviation

im = ax.imshow(data_sub, 
               interpolation='nearest', # 'nearest' gives us the ability to work with scaled up images. Each pixel is shows as a square of multiple pixels. 
               cmap='gray', #cmap = 'gray' uses gray color map. 
               vmin=m-s, vmax=m+s, origin='lower') #vmin = m-s will map the color scale linearly with appropriate color corresponding to vmax and vmin.

#individually plot around all of the objects
for i in range(len(objects)): #for all i between 0 and number of objects
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]), 
                width=6*objects['a'][i], # width is semi-major axis of the ellipse
                height=6*objects['b'][i], # Height is semi-minor axis of ellipse
                angle=objects['theta'][i] * 180. / np.pi) # Angle is theta in degrees
    e.set_facecolor('none') # 'none' means no ellispses found around our objects will be filled in
    e.set_edgecolor('red') # red ellispes
    ax.add_artist(e)       # Adds the ellipse subplot to the image subplot for an entire image
    
#Save the figure as a PNG in the local folder
plt.savefig('fig4.png',bbox_inches='tight', dpi=300) 

### Aperture photometry - Perform simple circular aperture photometry with a 3 pixel radius at the locations of the objects

In [ ]:
# We now need to Calculate the Flux
# Flux = total energy croossing an per unit time 
# Fluxerr = flux error; (sigma^2)_F = sigma^2 summed over i + F/g
# err=bck.globalrms is the rms fluctuation in the noise of the image 
# Set F = 3.0; sum of the flux within the radius of 3 pixels and aperture of around 0.4 arcseconds
# gain is the position uncertainty

flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'],
                                     3.0, err=bkg.globalrms, gain=1.0) 



In [ ]:
# Now we print the first 10 fluxes with theior respective uncertainties
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i])) 

### Plot the histogram of the fluxes

In [ ]:
 # Convert the flux data array to 1D
# Plot the Histogram of the fluxes. 
# X-axis - magnitude of the flux
# Y-axis - Number of objects with similar flux
# bins  - Number of rectangles the data is split into. Auto value will choose best fit.

histogram = plt.hist(flux.flatten(), bins='auto')
